In [3]:
# arff used to read arff file
import arff
# numpy used for turning list into np array
import numpy as np
# pandas for creating and manipulating dataframes
import pandas as pd
# math for basic calculations
import math

In [4]:
def prepare_data(filename):
    """
    Function to transform arff file to pandas dataframe that can be manipulated easily
    Only parameter needed is the accessible file name
    """
    # load arff file
    data = arff.load(open(filename))
    # store arff data as numpy array
    data_array = np.array(data['data'])
    # create dataframe
    dataset = pd.DataFrame(data_array)
    # add column names
    dataset.columns = ['cap shape', 'cap surface', 'cap color', 'bruises', 'odor', 'gill attachment', 'gill spacing', 'gill size', 'gill color',
             'stalk shape', 'stalk root', 'stalk surface above ring', 'stalk surface below ring', 'stalk color above ring',
              'stalk color below ring', 'veil type', 'veil color', 'ring number', 'ring type', 'spore print color', 'population','habitat', 'mushroom class']

    return dataset

In [5]:
def get_unique_count(target_column, return_counts = False):
    """
    Get unique count of an element in first parameter target column
    If return_count is true then return the count of each unique element
    Sort of copies numpy's unique() function but not as elegantly 
    """
    # empty list for unique elements
    unique_elements = []
    # append unique elements found in target_column into list
    for i in target_column:
        if i not in unique_elements:
            unique_elements.append(i)
    
    # store as numpy array
    unique_elements = np.array(unique_elements)
    
    # check if we should return element count as well
    if return_counts == True:
        counter = 0
        counts = [0] * len(unique_elements)
        
        # count elements
        for z in unique_elements:
            for i in target_column:
                if i == z:
                    counts[counter] += 1
            counter += 1

        counts = np.array(counts)
        return unique_elements, counts
    else:
        return unique_elements

In [6]:
def get_max(array):
    """
    Find index of highest number in array
    """
    current_max = -10000
    max_index = 0;
    
    for i in range(len(array)):
        if array[i] > current_max:
            max_index = i
            current_max = array[i]
            
    return max_index

In [7]:
def get_sum(array):
    """
    Return the sum of an array
    """
    sum = 0
    for item in array:
        sum += item
    return sum

In [8]:
def calculate_entropy(feature_column):
    """
    Calculate the entropy of a particular feature column.
    Only parameter is a feature column 
    """
    # return the unique elements in target column and return the count of them, both come back as numpy array
    unique_values, value_counts = get_unique_count(feature_column, return_counts = True)
    # incase of 'mushroom class' feature it will return:
    # unique_values = [e, p]
    # value_counts   = [3916, 4208]    
        
    # calculate entropy
    # loops over all the elements i and sequentially adds them in formula
    entropy = 0
    for i in range(len(unique_values)):
        item_count = (value_counts[i] / get_sum(value_counts))
        entropy += (-item_count) * math.log(item_count, 2) 
    return entropy

In [9]:
def calculate_information_gain(dataset, split_feature_name, target_feature_name = "mushroom class"):
    """
    Calculate the information gain of a dataset. This function takes a few parameters:
    - dataset = The dataset needed to calculate the information gain of the feature
    - split_feature_name = The name of the feature needed to calculate information gain for.
    - target_feature_name = The name of the target feature. The default for this is "mushroom class"
      because that is what i named the column.
    """
    
    # first we get the entropy of the total target feature
    total_target_entropy = calculate_entropy(dataset[target_feature_name])
    
    # next we get the values and the value_counts for the split attributes
    # as well as dropping any rows which are empty so count is not effected
    values, value_counts = get_unique_count(dataset[split_feature_name].dropna(), return_counts = True)
    
    # calculate the total entropy of the split feature
    total_split_entropy = 0
    for i in range(len(values)):
        # get item count   
        item_count = (value_counts[i] / get_sum(value_counts))
        
        # find next value in feature to calculate while removing the feature being selected
        # so it doesnt stay in our feature space
        next_feature_value = dataset.where(dataset[split_feature_name] == values[i]).dropna()[target_feature_name]

        # add entropy of feature value to total 
        total_split_entropy += item_count * calculate_entropy(next_feature_value)
    
    # get the information gain
    information_gain = total_target_entropy - total_split_entropy
    
    return information_gain

In [10]:
def build_tree(dataset, original_dataset, feature_space, target_feature_name = "mushroom class", root_node = None):
    """
    Main function to build tree using ID3 algorithm. It has several parameters:
    - dataset = the dataset which the ID3 algorithm will run on, at program start it will be the entire
        training dataset, but later on it could empty
    - original_dataset = Needed to get the mode of the target feature value of the original dataset in
        case the dataset delivered by the first parameter is empty (entire training dataset)
    - features = The feature space of the dataset. This will be used for the recursive call since during
        the tree growing process we have to remove features from our dataset when splitting nodes
    - target_feature_name = the name of the target attribute
    - parent_node = The parent node when running recursive calls to grow the tree
    """
    
    ## Creating Leaf Nodes:
    
    # only one unique value so we simply return it
    unique_target_values = get_unique_count(dataset[target_feature_name])
    if len(unique_target_values) == 1:
        return unique_target_values[0]
    
    # when the dataset is empty, we find the mode value in the target feature of the original dataset and return it  
    elif len(dataset) == 0:
        # get unique values from target column in original dataset
        unique_original_data, unique_counts = get_unique_count(original_dataset[target_feature_name], return_counts = True)
        # find index of highest unique value
        index_of_target_mode = get_max(unique_counts)
        # index_of_target_mode = get_max(get_unique_count(original_dataset[target_feature_name], return_counts = True)[1])
        return unique_original_data[index_of_target_mode]
    
    # when there are no features in the feature space simply return the current parent node
    elif len(feature_space) == 0:
        return root_node
    
    else:
        # set the parent node as the highest count value from the target feature 
        unique_data_elements = get_unique_count(dataset[target_feature_name])
        index_of_target_mode = get_max(get_unique_count(dataset[target_feature_name], return_counts = True)[1])
        root_node = unique_data_elements[index_of_target_mode]
        # parent node will equal to 'e' from 'mushroom class' in first run

        # store information gain of each feature in current feature space 
        info_gain_values = []
        for feature in feature_space:
            info_gain_values.append(calculate_information_gain(dataset, feature, target_feature_name))
        
        
        # locate highest valued feature stored in info_gain_values
        best_gain_feature_index = get_max(info_gain_values)
        best_gain_feature = feature_space[best_gain_feature_index]
        
        # create the tree structure. The root gets the name of the feature (best_gain_feature) with
        # the maximum information gain in the first run ---> 'odor'

        current_tree = {best_gain_feature: {} }

        # remove the feature with the best information gain from the feature space
        features = []
        for feature in feature_space:
            if feature != best_gain_feature:
                features.append(feature)


        # create a node for each of the unique values avialable in our best feature
        for value in get_unique_count(dataset[best_gain_feature]):
            unique_value = value
            
            # split the dataset along the value of the feature with the largest information gain 
            # and there we create sub_datasets to continue building our tree
            # we also remove any rows with missing values
            split_dataset = dataset.where(dataset[best_gain_feature] == unique_value).dropna()
            
            # call the ID3 algorithm for each of those sub_datasets with the new parameters --> here the recursion comes in
            split_tree = build_tree(split_dataset, dataset, features, target_feature_name, root_node)
            
            # add the split tree, grown from the sub_dataset to the tree under the root node
            current_tree[best_gain_feature][unique_value] = split_tree

            print(current_tree)
            
        return current_tree

In [11]:
def predict(observation, tree):
    """
    
    """
    # the key here is feature name
    for key in list(observation.keys()):
        # if we find that both our tree and observation contain this key
        if key in list(tree.keys()):
            # we try to store the resulting key value combo  
            try:
                result = tree[key][observation[key]] 
            # otherwise return a one
            except:
                return 1.0
            
            result = tree[key][observation[key]]
            # check to see if result is a dictionary
            if isinstance(result, dict):
                # if so we run it again so we can find the value
                return predict(observation,result)
            else:
                return result
            

In [12]:
def train_test_split(dataset):
    # using around ~30% (2500) of the data as training data gives a ~75% accuracy measure
    # while using ~80% (6500) will give ~55% accuracy
    # this looks like overfitting so i might try to find a solution for it
    # i chose to stick to 75% accuracy!
    
    # by resetting the index and dropping the old index we make sure the old index
    # doesn't transfer to the new datasets we create and we guarantee no troublesome errors
    train_data = dataset.iloc[:2500].reset_index(drop=True)
    test_data = dataset.iloc[2500:].reset_index(drop=True)
    
    return train_data, test_data

In [23]:
def k_fold(dataset, index, k):
    length = len(dataset)
    return dataset[length*(index-1)//k:length*index//k]

In [1]:
def get_accuracy(testing_data, tree):
    # remove mushroom class and form new list like observations
    observations = testing_data.iloc[:,:-1].to_dict(orient = "records")

    #Create an empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(testing_data)):
        predicted.loc[i,"predicted"] = predict(observations[i],tree) 
    
    # check correctly predicted column with mushroom class column then multiply 
    accuracy = (get_sum(predicted['predicted'] == testing_data['mushroom class']) / len(testing_data))
    return accuracy
    

In [35]:
# store data in dataframe
dataset = prepare_data('mushroom.arff')

# split the dataset into training and testing
training_data, testing_data = train_test_split(dataset)

# we remove the 'mushroom class' feature when adding the feature space
without_mushroom_class = training_data.columns[:-1]

# create tree
print("Building Tree: ")
tree = build_tree(training_data, training_data, without_mushroom_class)

print("\nFinal Tree:")
print(tree)

# test accuracy of tree built
print("\nAccuracy:")
accuracy = get_accuracy(testing_data,tree)
print(round(accuracy * 100, 4), "%")


Building Tree: 
{'odor': {'p': 'p'}}
{'odor': {'p': 'p', 'a': 'e'}}
{'odor': {'p': 'p', 'a': 'e', 'l': 'e'}}
{'odor': {'p': 'p', 'a': 'e', 'l': 'e', 'n': 'e'}}
{'odor': {'p': 'p', 'a': 'e', 'l': 'e', 'n': 'e', 'f': 'p'}}
{'odor': {'p': 'p', 'a': 'e', 'l': 'e', 'n': 'e', 'f': 'p', 'c': 'p'}}
<class 'pandas.core.frame.DataFrame'>
<class 'dict'>

Final Tree:
{'odor': {'p': 'p', 'a': 'e', 'l': 'e', 'n': 'e', 'f': 'p', 'c': 'p'}}

Accuracy:
76.7425 %


In [28]:
folds = []
folds.append(fold_i_of_k(dataset, 1, 3))
folds.append(fold_i_of_k(dataset, 2, 3))
folds.append(fold_i_of_k(dataset, 3, 3))

In [43]:
# fold 1 = training
# fold 2 = testing
foldsOne = folds[0].reset_index(drop=True)
foldsTwo = folds[1].reset_index(drop=True)
foldsThree = folds[2].reset_index(drop=True)

foldsOneTwo = foldsOne.append(foldsTwo, ignore_index=True)
foldsOneThree = foldsOne.append(foldsThree, ignore_index=True)
foldsTwoThree = foldsTwo.append(foldsThree, ignore_index=True)

treeOne = build_tree(foldsOneTwo, foldsOneTwo, foldsOneTwo.columns[:-1])
treeTwo = build_tree(foldsOneThree, foldsOneThree, foldsOneThree.columns[:-1])
treeThree = build_tree(foldsTwoThree, foldsTwoThree, foldsTwoThree.columns[:-1])

accuracyOne = get_accuracy(foldsThree, treeOne)
accuracyTwo = get_accuracy(foldsTwo, treeTwo)
accuracyThree = get_accuracy(foldsOne, treeThree)

print(round(accuracyOne * 100, 4), "%")
print(round(accuracyTwo * 100, 4), "%")
print(round(accuracyThree * 100, 4), "%")

{'odor': {'p': 'p'}}
{'odor': {'p': 'p', 'a': 'e'}}
{'odor': {'p': 'p', 'a': 'e', 'l': 'e'}}
{'spore print color': {'n': 'e'}}
{'spore print color': {'n': 'e', 'k': 'e'}}
{'spore print color': {'n': 'e', 'k': 'e', 'r': 'p'}}
{'cap color': {'c': 'e'}}
{'cap color': {'c': 'e', 'n': 'e'}}
{'cap color': {'c': 'e', 'n': 'e', 'w': 'p'}}
{'spore print color': {'n': 'e', 'k': 'e', 'r': 'p', 'w': {'cap color': {'c': 'e', 'n': 'e', 'w': 'p'}}}}
{'odor': {'p': 'p', 'a': 'e', 'l': 'e', 'n': {'spore print color': {'n': 'e', 'k': 'e', 'r': 'p', 'w': {'cap color': {'c': 'e', 'n': 'e', 'w': 'p'}}}}}}
{'odor': {'p': 'p', 'a': 'e', 'l': 'e', 'n': {'spore print color': {'n': 'e', 'k': 'e', 'r': 'p', 'w': {'cap color': {'c': 'e', 'n': 'e', 'w': 'p'}}}}, 'f': 'p'}}
{'odor': {'p': 'p', 'a': 'e', 'l': 'e', 'n': {'spore print color': {'n': 'e', 'k': 'e', 'r': 'p', 'w': {'cap color': {'c': 'e', 'n': 'e', 'w': 'p'}}}}, 'f': 'p', 'c': 'p'}}
{'odor': {'p': 'p', 'a': 'e', 'l': 'e', 'n': {'spore print color': {'n':